In [ ]:
# Module Import
import numpy as np
import cv2

############### 함수 정의 ######################

# 모서리 네 개로 관심영역을 선택하는 함수
def drawROI(img, corners): # corners는 scrQuad좌표 (초기화 좌표값)
    img_pnt = img.copy()
    
    # 모서리 수 만큼 원 생성. corners (초기화된 좌표값) 정보 이용
    for i in corners:
        cv2.circle(img_pnt, tuple(i.astype(int)), 10, black, -1, cv2.LINE_AA)
    
    # 각 모서리를 잇는 선.
    cv2.line(img_pnt, tuple(corners[0].astype(int)), tuple(corners[1].astype(int)), black, 2, cv2.LINE_AA)
    cv2.line(img_pnt, tuple(corners[1].astype(int)), tuple(corners[2].astype(int)), black, 2, cv2.LINE_AA)
    cv2.line(img_pnt, tuple(corners[2].astype(int)), tuple(corners[3].astype(int)), black, 2, cv2.LINE_AA)
    cv2.line(img_pnt, tuple(corners[3].astype(int)), tuple(corners[0].astype(int)), black, 2, cv2.LINE_AA)
    
    # addWeighted를 이용해서 입력 영상과 img_pnt영상에 가중치를 적용하여 투명도 적용
    disp = cv2.addWeighted(img, 0.5, img_pnt, 0.5, 0)
    
    return disp

# 마우스 이벤트) 원근변환 함수
def perspective (event, x, y, flags, param):
    global pts1, dragSrc, img, ptOld
    
    if event == cv2.EVENT_LBUTTONDOWN:
        for i in range(4):
            # 클릭한 점이 원 안에 있는지
            if cv2.norm(pts1[i] - (x, y)) < 25:
                dragSrc[i] = True
                ptOld = (x, y) # 마우스를 이동할 때 모서리도 따라 움직이도록 설정
                break
                
    if event == cv2.EVENT_LBUTTONUP:
        for i in range(4):
            dragSrc[i] = False
            
    if event == cv2.EVENT_MOUSEMOVE:
        for i in range(4):
            if dragSrc[i]: # True일 경우
                dx = x - ptOld[0] # 이전의 마우스 점에서 dx, dy 만큼 이동
                dy = y - ptOld[1]
                
                pts1[i] += (dx, dy) # 이동한 만큼 더해줌
                
                img_pnt = drawROI(img, pts1)
                cv2.imshow('img', img_pnt) # 수정된 좌표로 모서리 이동
                ptOld = (x, y) # 현재 점으로 설정
                break

# 이미지 전처리 함수 (erode & dilate)
def preprocessing(threshold, kernel, cnt):
    se = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel, kernel))

    for i in range(cnt):
        erosion = cv2.erode(threshold, se)
       
    dilation = cv2.dilate(erosion, se)
    
    return dilation

# 훼손 정도 %로 측정 함수
def extract_damage(img_subtract):
    '''
    훼손 픽셀 : 255 - 0 = 255
    비훼손 픽셀 : 255 - 255 = 0
    '''
    
    global cnt_white, cnt_black
    
    for i in img_subtract: # 열
        for j in i: # 행
            if j == 255:
                cnt_white += 1
            if j == 0:
                cnt_black += 1
                
    # print('훼손:', cnt_white)
    # print('비훼손:', cnt_black)
    # print(cnt_white + cnt_black)
    print('횡단보도의 훼손 정도: {:.2f} % '.format(cnt_white/(cnt_black + cnt_white) * 100, '%'))
    
    cnt_white = 0 # 훼손 정도 출력 후 개수 초기화
    cnt_black = 0

# 마우스 이벤트) 드래그로 관심영역 지정 함수
def onROI (event, x, y, flags, param):
    global isDragging, x0, y0, result, roi, cnt_black, cnt_white
    
    # LBUTTONDOWN
    if event == cv2.EVENT_LBUTTONDOWN:
            
        isDragging = True
        
        x0 = x
        y0 = y
    
    # MOUSEMOVE
    elif event == cv2.EVENT_MOUSEMOVE:
        if isDragging:
            
            # 사각형을 그리는 과정을 표시할 buffer 생성.
            buffer = result.copy()
            cv2.rectangle(buffer, (x0, y0), (x, y), white, 1)
            cv2.imshow('result', buffer)
            
    # LBUTTONUP (1. 클릭하는 경우 2. 드래그하는 경우)
    elif event == cv2.EVENT_LBUTTONUP:
        
        # 현 좌표 - 최초 좌표 = w, h
        w = x - x0
        h = y - y0
        
        # 클릭
        # lbuttondown, lbuttonup이 정확한 한 픽셀에서 선택되기 어려우므로 여유를 둠.
        if abs(w) < 3 or abs(h) < 3:
            isDragging = False

            # roi가 없는 경우 roi 생성을 위한 드래그 유도.
            if roi is None:
                print('왼쪽 상단에서 오른쪽 하단으로 드래그 하시오.')
                
            # roi가 있는 경우 roi를 출력.
            else:
                
                # 드래그한 roi를 출력
                cv2.imshow('roi', roi)
                # 훼손 정도 측정하기.
                # 1. roi와 같은 크기의 흰색 이미지 생성
                img_like = np.full_like(roi, 255)
                drag_subtract = img_like - roi
                cv2.imshow('drag_subtract', drag_subtract)
                cv2.imshow('img_like', img_like)
        
                # 훼손 정도 계산.
                extract_damage(drag_subtract)
        
        # 드래그
        else :
            isDragging = False
            roi = result[y0:y, x0:x]
    
    # RBUTTONDOWN
    elif event == cv2.EVENT_RBUTTONDOWN:
        
        cv2.destroyWindow('roi')
        cv2.destroyWindow('img_like')
        cv2.destroyWindow('drag_subtract')
        
        # 원근 변환된 이미지 이진화
        t, t_otsu = cv2.threshold(dst, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        result = preprocessing(t_otsu, 3, 2)

        # 결과 영상 출력
        cv2.imshow('result', result) # 원근 변환 + 이진화 + 전처리 이미지.

# 라벨링 & 도형 검출 함수
def label (preprocessed_img):
    
    img_like = np.full_like(result, 0) # 전처리된 이미지와 같은 크기의 뺄셈용 검정 이미지 생성
        
    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(result)

    # 연산에 사용할 두 이미지에 사각형 출력하기.
    for i in range(1, cnt):
        (x, y, w, h, area) = stats[i]
        cv2.rectangle(result, (x, y, w, h), white) # 전처리된 이미지에 사각형 출력
        cv2.rectangle(img_like, (x, y, w, h), white, -1) # 뺄셈용 이미지에 흰색 사각형 채우기로 출력
        cv2.imshow('result', result)
        cv2.imshow('img_like', img_like)
        
    # img_like와 result를 빼기.
    subtract = img_like - result
    # 빼기 연산을 실행한 결과 이미지 출력.
    cv2.imshow('subtract', subtract)
    
    # 함수 호출) extract_damage (훼손 정도 %로 측정 함수)
    extract_damage(subtract)
    
############### 변수 초기화 ######################

# 이미지 불러오기
img = cv2.imread('img/hw7.jpg', cv2.IMREAD_GRAYSCALE)

height, width = img.shape[:2] # 입력 영상 크기
height = round(height/10) # 입력 영상 크기 조절
width = round(width/10)

img = cv2.resize(img, dsize=(height*2, width*2), interpolation=cv2.INTER_AREA)

dheight = height # 출력 영상 크기
dwidth = width

isDragging = False
x0, y0, w, h = -1, -1, 0, 0
white = (255, 255, 255)
black = (0, 0, 0)
roi = None
cnt_black = 0 # 비훼손 픽셀 개수
cnt_white = 0 # 훼손 픽셀 개수

# 초기 모서리 위치. (30은 임의의 초기화 좌표값)
pts1 = np.array([[30, 30], [30, height-30], [width-30, height-30], [width-30, 30]], np.float32)
# 출력 모서리 위치.
pts2 = np.array([[0, 0], [0, dheight-1], [dwidth-1, dheight-1], [dwidth-1, 0]], np.float32)
# 드래그 상태 여부
dragSrc = [False, False, False, False]

############### 프로그램 실행 ######################

# 함수 호출) drawROI (모서리 네 개로 관심영역을 선택하는 함수)
disp = drawROI(img, pts1)

# 결과 영상 출력
cv2.imshow('img', disp)

# 마우스콜백함수 등록) perspective (원근 변환 함수)
cv2.setMouseCallback('img', perspective)

while True:
    if cv2.waitKey() == 13: # ENTER 키
        break

# 원근 변환행렬 생성
pers = cv2.getPerspectiveTransform(pts1, pts2)
# 원근 변환 적용
dst = cv2.warpPerspective(img, pers, (dwidth, dheight), flags=cv2.INTER_CUBIC)

# 원근 변환된 이미지 이진화
t, t_otsu = cv2.threshold(dst, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 함수 호출) preprocessing (이미지 전처리)
result = preprocessing(t_otsu, 3, 2)

# 결과 영상 출력
cv2.imshow('dst', dst) # 원근 변환된 원본 이미지.
cv2.imshow('result', result) # 원근 변환 + 이진화 + 전처리 이미지.

# 마우스콜백함수 등록) onROI (드래그로 관심영역 지정 함수)
cv2.setMouseCallback('result', onROI)
        
# 라벨링, 도형 검출
while True:
    if cv2.waitKey() == 13: # ENTER 키
        
        # 함수 호출) label (라벨링 & 도형 검출 함수)
        label(result)
        
    if cv2.waitKey() == 8: # BACKSPACE 키
        # 함수 호출) preprocessing (이미지 전처리)

        cv2.destroyWindow('img_like')
        cv2.destroyWindow('subtract')
        
        result = preprocessing(t_otsu, 3, 2)

        # 결과 영상 출력
        cv2.imshow('result', result) # 원근 변환 + 이진화 + 전처리 이미지.
        
    if cv2.waitKey() == 27: # ESC 키를 누르면 프로그램 종료.
        break
        
cv2.destroyAllWindows()